In [1]:
from text_dedup.minhash import embed_func
from dedup_bindings import EmbedFunc
import datasets as ds

In [2]:
Emb = EmbedFunc(threshold =0.5, num_perm=200, false_positive=0.5, false_negative=0.5)

In [4]:
Emb.embed_func(text="Lorem ipsum dolor sit amet, consectetur adipiscing elit")

thread '<unnamed>' panicked at /rustc/9b00956e56009bab2aa15d7bff10916599e3d6d6/library/core/src/num/mod.rs:1157:5:
attempt to multiply with overflow
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: attempt to multiply with overflow